In [5]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from pprint import pprint as pp

### The available pickle objects are dictionaries with three dataframes, one for each type of event counted
```
df_all = {
    'cases': df_from_csv('csv/confirmed_cases.csv'),
    'deaths': df_from_csv('csv/deaths.csv'),
    'recovered': df_from_csv('csv/recovered_cases.csv')
}
```

In [6]:
df_all = pickle.load(open('../pickles/df_c19all.p', 'rb'))
df_cases = df_all['cases']
# ...